In [1]:
import os
import bs4
import regex
import shutil
import zipfile
from bs4 import BeautifulSoup

In [2]:
# from txt_to_dataset import find_characters
# from txt_to_dataset import find_author
# from txt_to_dataset import find_end

---

In [3]:
TXT_DIR = "theatregratuit-txt"
off = "\t"
if os.path.isdir(TXT_DIR):
    fnames = [x for x in os.listdir(TXT_DIR) if ".txt" == os.path.splitext(x)[-1]]
    n_files = len(fnames)

In [4]:
tmp_dir = "tmp1"
if not os.path.isdir(tmp_dir):
    os.mkdir(tmp_dir)

---

In [5]:
def printlines(lines):
    print('\n'.join(lines))

In [6]:
def printsep(s):
    print(s*40)

In [7]:
def underprint(x):
    print(x)
    print('-'*len(x))

In [8]:
def print_index_split(lines, index):
    printlines(lines[:index])
    printsep('*')
    printlines(lines[index:index+10])

In [9]:
def print_file(f, whole_file=False):
    underprint(f)
    print()
    raw, lines, lines_len = get_lines(f)
    if whole_file:
        printlines(lines)
    else:
        lines_shortened, _ = shortened_lines(f, lines, lines_len)
        printlines(lines_shortened)  

---

In [457]:
def get_lines(input_name):
    with open(os.path.join(TXT_DIR, input_name), "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
        lines = [l.strip() for l in raw.split('\n')]
    return raw, lines, len(lines)

In [1087]:
def shortened_lines(fname, lines, lines_len):
    author_index = find_author(fname, lines, lines_len) 
    if not author_index:
        author_index = index_of_regex_match(lines, REGICES["additional_author"])
    char_index = find_characters(fname, lines, lines_len)        
    end_index = find_end(fname, lines, lines_len)
    # print(fname, char_index, author_index, end_index)
    start_index = max(author_index, char_index)
    return lines[start_index:end_index], {
        "char_index": char_index, 
        "author_index": author_index, 
        "end_index": end_index,
        "start_index": start_index
    }

In [1604]:
def end_of_block_and_trim(lines, lines_len, j, k):
    # find end of block
    while j+k < lines_len and not regex.match(REGICES["blank_line"], lines[j + k]):
        k += 1
    # trim empty lines
    while j+k < lines_len and regex.match(REGICES["blank_line"], lines[j + k]):
        k += 1     
    return k

In [1665]:
def find_characters(fname, lines, lines_len):  
    feydeau = False
    if fname in (
        "count-2047890-LE_ROI.txt",
        "count-3031455-Nono.txt",
        "count-1712853-LE_VERITABLE_SAINT_GENEST.txt",
        "count-3263112-LA_MORT_DE_WALLENSTEIN.txt",
        "count-3397657-LA_VEINE.txt",
        "count-2158361-ANDROMEDE.txt",
        "count-2037607-LHabit_vert.txt",    
    ):
        feydeau = True
    vega = False
    char_index = 0
    found_char = False
    limit = 15
    
    if fname == "count-1537913-LES_ESPAGNOLS_EN_DANEMARK.txt":
        limit = 26
    for j, l in enumerate(lines[:limit]):
        if found_char:
            break
        if "Feydeau" in l and fname not in (
            "count-2199212-Les_Paves_de_lours.txt",
            "count-2316496-On_va_faire_la_cocotte.txt",            
        ):
            feydeau = True
        if "Lope de Vega" in l:
            vega = True
        if regex.match(REGICES["character"], l):
            found_char = True
            k = 1
            # files with space after char > k + 1
            if fname in (
                "count-2158361-ANDROMEDE.txt",
                "count-2037607-LHabit_vert.txt",
                "count-1182649-LE_MONDE_OU_L.txt",
                "count-1037209-LES_MAMELLES_DE_TIRESIAS_-_Guillaume_Apollinaire.txt",
                "count-958980-Le_gendre_de_Monsieur_Poirier_-_Emile_Augier.txt"                
            ):
                k += 1
            # more annoying exceptions
            if fname in ("count-959005-LOrphelin_de_la_Chine_-_Voltaire.txt",):
                # find init caps words only, no space (lest we go to the end)
                while j+k < lines_len and regex.match(REGICES["caps_word"], lines[j + k]):
                    k += 1    
            elif vega: 
                # all vega can go up to "La scène"
                while j+k < lines_len and not regex.match(REGICES["la_scene"], lines[j + k]):
                    k += 1                
            else:
                k = end_of_block_and_trim(lines, lines_len, j, k)
                # annoying exceptions: more chars after blank line
                if feydeau:
                    k = end_of_block_and_trim(lines, lines_len, j, k)
                    # some files like Feydeay
                    if fname in ("count-3263112-LA_MORT_DE_WALLENSTEIN.txt",
                                 "count-3397657-LA_VEINE.txt"):
                        k = end_of_block_and_trim(lines, lines_len, j, k)
            char_index = j + k          
    return char_index

In [1430]:
def find_author(fname, lines, lines_len):
    author_index = 0
    found_author = False
    for j, l in enumerate(lines[:10]):
        if found_author:
            break
        if regex.match(REGICES["author"], lines[j]):
            found_author = True
            # annoying exceptions
            if fname in ("count-1049195-Arret_36_de_lautobus_40_-_Jean_Sibil.txt",
                         "count-1396213-Roberto_Succo.txt"):  
                k = 1
            elif fname == "count-1541294-Olaf_loriginal.txt":
                k = 2 
            else:
                k = 2  # we assume "de\n\nauthor"
                while j + k < lines_len and not regex.match(
                    REGICES["blank_line"], lines[j + k]
                ):
                    k += 1
            author_index = j + k + 1
    return author_index

In [13]:
def find_end(fname, lines, lines_len):
    end_index = lines_len - 1
    found_end = False
    for j, l in enumerate(reversed(lines[-4:])):
        if found_end:
            break
        if regex.search(REGICES["fin"], l):
            found_end = True
            end_index = end_index - j - 1
    return end_index

In [195]:
def index_of_regex_match(lines, r, trim=True):
    lines_len = len(lines)
    ind = 0
    found = False
    for i,l in enumerate(lines):
        if found: break
        if regex.search(r, l):
            found = True
            k = 1            
            if trim: 
                while i + k < lines_len and \
                 regex.match(REGICES["blank_line"], lines[i + k]):
                    k += 1
            ind = i + k
            break
    return ind 

In [709]:
def file_data(f, verbose=True):
    if verbose:
        underprint(f)
        print()
    raw, lines, lines_len = get_lines(f)
    lines_shortened, indices = shortened_lines(f, lines, lines_len)
    lines_shortened_len = len(lines_shortened)
    data = {
        "fname": f,
        "raw": raw,
        "lines": lines,
        "lines_len": lines_len,
        "lines_shortened": lines_shortened,
        "lines_shortened_len": lines_shortened_len
    }
    data.update(indices)
    return data

In [1220]:
def make_regices():
    return {
        "blank_line": regex.compile("^\s*$"),
        "blank_line_with_rubbish": regex.compile("^[\p{Z}\p{P}]*$"),
        "character": regex.compile(
            "^((les)*pe*rsonna *ge|(les)*acteurs|dramatis personae|avertissement|entreparleur|biographies|apparences|personrage|persongueules|pépersonâge)",
            regex.IGNORECASE,
       ),
        "author": regex.compile("^(de|" +\
                                 "\(Auteur inconnu\))$",
                                regex.IGNORECASE),
        "additional_author": regex.compile("^(Voltaire|" +\
                                           "de\s+Georges Courteline|" +\
                                           "Translation en prose de Jean Sibil|" +\
                                           "Anton Pavlovitch Tchekhov|" +\
                                           "EUGENE LABICHE, A. LEFRANC et MARC-MICHEL|" +\
                                           "Alexandre Hardy)$"),
        "fin": regex.compile("(fin|rideau|f1n|inachev|manque)", regex.IGNORECASE),
        # search for lines made of full caps + punct/space, as well as some
        # very common words listing characters, and didascalia in ()
        "caps_full": regex.compile("^\s*((\p{Lu}+|seule?|puis|puis tout le monde|moins|et)\s*[.’'<>;,]*\s*)+\s*(\(.*?\))*\s*$"),
        # common annoying starts "M.", "L'"
        "annoying_init": regex.compile("^[\p{P}\p{Z}]*(M\.|\p{Lu}')\p{Z}*"),
        # words (with possible - or '), and space, with punctuation at the end
        "caps_init": regex.compile("^\s*(([\p{Lu}\p{Pc}\p{Pd}1]+\p{Zs}*)+)(\s*[\p{Po}]\s*)"),
        "char_lc_dot_dash": regex.compile("^(\p{Z}*M*[\p{Ll}\p{Z}]+\.)\p{Z}*\p{Pd}+\p{Z}*(.*)$"),
        "caps_word": regex.compile("\p{Lu}{2,}"),
        "trailing_space": regex.compile("\s*$"),
        "trailing_punct": regex.compile("\s*[.,:]*\s+$"),
        "non_breaking_space": regex.compile(" {2,}"), # non-breaking space
        "M." : regex.compile("M.\s"),
        "dot": regex.compile("\s*\.\s*"),
        "final_dot": regex.compile("\s*\.\s*$"),
        "final_comma": regex.compile(",$"),
        # include rare errors like ".-—Blah" or ". -— Blah"
        "dash": regex.compile("[,.](\s*[-–—]+|-[–—])\s*"),
        "dash_and_more": regex.compile("(.*?)[.;:,]*\s*-*[-–—]\s*"),
        "colon": regex.compile("\s*:\s*"),
        "colon_and_more": regex.compile("(.*?)\s*:\s*"),
        "didasc_and_more": regex.compile(".*?(?<!M)[.:]\s*"),
        # "char_no_dot": regex.compile("^\s*([A-Z1'-]+)\s*$"),
        # # using all 3 lengths: -, –, —, bc of inconsistencies
        # "char_dash": regex.compile("^\s*([A-Z1'-]{2,})\.[\s-]+[–—]\s"),
        # "char_comma_dash": regex.compile("^\s*([A-Z1'-]{2,},.*?)\.[\s-]+[–—]\s"),
        # "char_dot_more": regex.compile("^\s*([A-Z1'-]{2,}\.)(.*)$"),
        # "char_no_dot_more": regex.compile("^\s*([A-Z1'-]{2,})(.*)$"),
        # "char_colon": regex.compile("^\s*([A-Z1'-]{2,})\s*:\s*$"),
        # "char_colon_more": regex.compile("^\s*([A-Z1'-]{2,})\s*:\s*(.*)$"),
        "first_scene": regex.compile("(sc[èe]ne (premi[èe]re|I)|premier tableau|I, 1\.)", regex.IGNORECASE),
        "act_or_scene": regex.compile("(acte|sc[èe]ne|^I\s*$)", regex.IGNORECASE),
        "la_scene": regex.compile("^La scène"),
        
        # block (multiline) regices (to act on raw text)
        "characters_block": regex.compile("\n*\s*PERSONNAGES?(.*?)(\n\s*\n)+", regex.DOTALL),
        
        # line cleanup
        "sc_ene": regex.compile("SC\p{Z}+[EÈ]NE", regex.IGNORECASE),
        "deuxi_eme": regex.compile("DEUXI\p{Z}ÈME", regex.IGNORECASE)
    }

REGICES = make_regices()

---

In [351]:
def split_by_regex(fnames, r, orig_file=False):
    with_r = []
    without_r = []
    for fname in fnames:
        raw, lines, lines_len = get_lines(fname)
        if not orig_file:
            lines, _ = shortened_lines(fname, lines, lines_len)
        found = False
        for i, l in enumerate(lines):
            if regex.search(r, l):
                found = True
                break
        if found:
            with_r.append(fname)
        else:
            without_r.append(fname)
    print("with:", len(with_r))
    print("without:", len(without_r))
    return with_r, without_r

---
### All files.

In [1601]:
fnames_it = iter(fnames)
ind = 0

In [1602]:
fname = next(fnames_it)
# fname = "count-1396213-Roberto_Succo.txt"
ind +=1 
underprint(fname)
print(ind)
print()
raw, lines, lines_len = get_lines(fname)
# printlines(lines)
lines_shortened, data = shortened_lines(fname, lines, lines_len)
# printlines(lines_shortened)
print_index_split(lines, max(data["char_index"], data["author_index"]))

count-2454059-Champignol_malgre_lui.txt
---------------------------------------
1



UnboundLocalError: local variable 'k' referenced before assignment

---
### Ending

Unproblematic (**hopefully**). `find_end` does the job, used in `shortened_lines`.

In [179]:
with_end, without_end = split_by_regex(fnames, REGICES["fin"])

with: 1083
without: 0


---
### Author

Split by formatted author: clean author files with formatted author name and files without.

In [352]:
auth_fname, no_auth_fnames = split_by_regex(fnames, REGICES["author"], orig_file=True)

with: 1070
without: 13


In [353]:
fnames_it = iter(no_auth_fnames)

In [364]:
fname = next(fnames_it)
raw, lines, lines_len = get_lines(fname)
author_index = index_of_regex_match(lines, REGICES["additional_author"], trim=True)
print_index_split(lines, author_index)


****************************************
Ruy Blas

1858

Personnages

RUY BLAS
DON SALLUSTE DE BAZAN
DON CÉSAR DE BAZAN
DON GURITAN


In [365]:
fnames_it = iter(auth_fname)

In [381]:
fname = next(fnames_it)
raw, lines, lines_len = get_lines(fname)
author_index = find_author(fname, lines, lines_len)
print_index_split(lines, author_index)

La Punaise

de

Vladimir Maïakovski

****************************************
Traduction de M. Wassiltchikov

PERSONNAGES :
PRISSIPKINE, PIERRE SKRIPKINE, ex-ouvrier, ex-membre du Parti; à présent, fiancé.
ZOIA BEREZKINE, ouvrière.
Famille RENAISSANCE :
ELZEVIRE DAVIDOVNA, fiancée, manucure, caissière du salon de coiffure.
ROSALIE PAVLOVNA, mère-coiffeuse.
DAVID OSSIPOVITCH, père-coiffeur.
OLEG BAYAN, autodidacte, ancien propriétaire.


Updating `find_author` & `shortened_lines` accordingly.

---
### Characters
After that, clean characters.

In [382]:
char, no_char = split_by_regex(fnames, REGICES["character"])

with: 889
without: 194


In [525]:
fnames_it = iter(char)
# fnames_it = iter(no_char)

In [534]:
fname = next(fnames_it)
print_file(fname, whole_file=True)

count-1355599-LE_CARROSSE_DU_SAINT.txt
--------------------------------------

LE CARROSSE DU SAINT-SACREMENT

Saynète

de

Prosper Mérimée

Tu veras que mis finezas
Te desenojan.
CALDERÔN
(Cual es la major perfection)

PERSONNAGES :
DON ANCRES DE RIBERA, vice-roi du Pérou .
L'EVEQUE DE LIMA.
LE LICENCIE TOMAS D‘ESQUIVEL.
MARTINEZ, secrétaire intime du vice-roi.
BALTHASAR, valet de chambre du vice-roi.
CAMILA PERICHOLE, comédienne.

La scène est à Lima, en 17...

Le cabinet du Vice-Roi.

Le VICE-ROI, en robe de chambre, assis dans un grand fauteuil auprès d'une table couverte de papiers. Une de ses jambes enveloppée de flanelle repose sur un coussin. Martinez debout auprès de la table, une plume à la main.

MARTINEZ : MM. les Auditeurs attendent la réponse de Votre Altesse.
LE VICE-ROI, d'un ton chagrin : Quelle heure est-il ?
MARTINEZ : Bientôt dix heures. Votre Altesse a justement le temps de s'habiller pour la cérémonie.
LE VICE-ROI : Le temps est beau, dis-tu ?
MARTINEZ : Oui, mons

Updating `find_character` & `shortened_lines`.

In [584]:
fnames_it = iter(fnames)

In [770]:
fname = next(fnames_it)
# fname = "count-959005-LOrphelin_de_la_Chine_-_Voltaire.txt"
# fname = "count-1049195-Arret_36_de_lautobus_40_-_Jean_Sibil.txt"
fname = "count-1541294-Olaf_loriginal.txt"
raw, lines, lines_len = get_lines(fname)
lines_shortened, indices = shortened_lines(fname, lines, lines_len)
print_index_split(lines, indices["author_index"] + indices["char_index"])

Olaf l'original

de

Jean Sibil
****************************************
(Catherine I)
Un salon-salle à manger au mobilier à la fois en harmonie et hétéroclite. Superbes rideaux. Il règne un certain laisser-aller : revues diverses en vrac dans un coin, coussins mal placés sur le divan, fauteuils mal placés (deux sont vis-à-vis pour étendre ses jambes sur l'autre)... Aux murs, très soigneusement accrochés, des photos de mode, des affiches de concerts exceptionnels (toutes musiques), des posters de films récents... Effet de surcharge.
Olaf (la cinquantaine sportive, tenue de footing, une petite queue de cheval et une mèche bleue sur le devant que, debout vers la cheminée, il examine dans une glace de poche) : Je sentais que l'orange me siérait mieux.
Corinne (la trentaine, avachie dans ses fauteuils vis-à-vis, en robe de chambre à grosses fleurs) : Chéri, je t'aime de toutes les couleurs.
Olaf : Tu t'en tires toujours comme ça au lieu de donner des conseils utiles.
Corinne (agressive) : 

---

### Space after "PERSONNAGES :"

In [1619]:
space_after_char = []
for f in fnames:
    raw, lines, lines_len = get_lines(f)
    if regex.search(r"\n\s*PERSONNAGES?\s*:*\n\s*\n", raw):
        space_after_char.append(f)
print(space_after_char)

['count-2158361-ANDROMEDE.txt', 'count-2037607-LHabit_vert.txt', 'count-1182649-LE_MONDE_OU_L.txt', 'count-1037209-LES_MAMELLES_DE_TIRESIAS_-_Guillaume_Apollinaire.txt', 'count-958980-Le_gendre_de_Monsieur_Poirier_-_Emile_Augier.txt']


---

### Checks for results with very few lines

In [1666]:
too_short = []
for f in fnames:
    data = file_data(f, verbose=False)
    if data["lines_shortened_len"] < 10:
        too_short.append(data)
len(too_short)

7

In [1667]:
too_short_it = iter(too_short)

In [1675]:
dn = next(too_short_it)
underprint(dn["fname"])
printlines(dn["lines_shortened"])
printsep('-')
printlines(dn["lines"])

StopIteration: 

---

In [1676]:
caps_after_chars = []
no_caps_after_chars = []
for fname in fnames:
    raw, lines, lines_len = get_lines(fname)
    lines_shortened, data = shortened_lines(fname, lines, lines_len)
    for l in lines_shortened:
        if regex.search(REGICES["caps_word"], l):
            caps_after_chars.append(fname)
            break
        else:
            no_caps_after_chars.append(fname)
            break              

In [1677]:
print(len(caps_after_chars))
print(len(no_caps_after_chars))

488
595


---

In [412]:
first_scene, no_first_scene = split_by_regex(fnames, REGICES["first_scene"], orig_file=True)

with: 732
without: 351


In [420]:
# fnames_it = iter(first_scene)
fnames_it = iter(no_first_scene)

In [429]:
fname = next(fnames_it)
print_file(fname, whole_file=True)

count-2478595-Le_Testament_du_Pere_Leleu.txt
--------------------------------------------

Le Testament du Père Leleu
Farce paysanne
 
de
 
Roger Martin du Gard
 
PERSONNAGES :
LA TORINE 
LE PERE LELEU 
LE PERE ALEXANDRE
LE NOTAIRE
(LE PERE Leleu et LE PERE Alexandre doivent être joués par le même acteur.)
 
Cette farce peut se jouer sur une estrade, sans décor. A gauche, un fauteuil de paysan; au milieu, une table de cuisine et deux chaises de paille; à droite, une longue huche à pain.
 
ACTE I
LE PERE ALEXANDRE est dans le fauteuil, enveloppé d'une couverture, un bonnet de coton sur la tête, le buste soulevé sur un oreiller, les bras étendus, le masque immobile. C'est un vieux, rasé, sec comme un os; l'œil, perçant et fixe, semble ne pas avoir de paupière. Les mots s'échappent de ses gencives sans dents, comme les sons d'une claquette de bois.
LA TORINE est une fille de trente ans, aux lèvres épaisses, à la poitrine molle, les cheveux serrés sous un bonnet. Elle est au fond de la scè

In [214]:
fname = next(fnames_it)
index += 1
underprint(f"{fname} | {index}")

raw, lines, lines_len = get_lines(fname)
lines_shortened, indices = shortened_lines(fname, lines, lines_len)
print("\t"*5 + "char/author/end indices:", indices, "| lines length:", lines_len)
print()

printlines(lines_shortened[:20])
# print()
# printsep('-')
# print()
# print_index_split(lines, indices[1])

count-2843271-Un_coup_de_tete.txt | 13
--------------------------------------
					char/author/end indices: (0, 7, 143) | lines length: 145

(Elle entre par le fond, l'air agité.)
 
C'est fait ! la lettre est envoyée.
Le dépit m'a donné du cœur,
Et de ma main je l'ai jetée
A la poste, sans avoir peur.
Oui, nous revenions de l'église
— Ma gouvernante allait devant —,
Nul ne me voyait, je l'ai mise
Dans la boîte, résolument !
Mon Dieu, j'ai mis bien peu de chose;
Ma lettre était si courte ! Quoi !
Ce n'est pas bien mal, je suppose ?
Trois mots : « Ernest, enlevez-moi ! »
« Enlevez-moi ! » Ça n'est pas grave;
Ça se fait dans tous les romans :
On voit toujours un seigneur brave,
Beau, galant, bref, plein d'agréments,
Enlever la belle qu'il aime,
Quand… Ah ! ce cas est trop commun !


---

In [536]:
act_or_scene, no_act_or_scene = split_by_regex(fnames, REGICES["act_or_scene"], orig_file=True)

with: 986
without: 97


In [558]:
# fnames_it = iter(act_or_scene)
fnames_it = iter(no_act_or_scene)

In [599]:
fname = next(fnames_it)
print_file(fname)

count-2740026-Jai_mal_aux_dents.txt
-----------------------------------

Vous me voyez ? Eh bien j'enrage. 
Figurez-vous qu'en cet instant 
J'aurais une femme, un ménage 
Si j'avais été bien portant !... 
J'arrive droit de la mairie... 
Mais patatras ! quel contretemps ! 
Le jour même où je me marie, 
C'était prévu ! j'ai mal aux dents.
 
C'est ma faute ! Avant hier dimanche, 
Avant de rompre tout à fait 
J'ai voulu dîner avec Blanche 
Et voici comment tout s'est fait : 
Nous étions seuls, en tête à tête, 
Chez Brébant, tous deux fort contents; 
Blanche, ô hasard ! n'était point bête; 
Moi... je n'avais point mal aux dents.
 
C'était au mieux ! comme on le pense. 
Nous avons fait un dîner fin : 
Bisque, huîtres, tout en abondance : 
Vin de Champagne et Chambertin, 
Rien n'y manquait ! c'était un rêve. 
Nous étions si gais, si bruyants ! 
Et de l'esprit, j'en eus sans trêve : 
Ah ! je n'avais pas mal aux dents.
 
Au dessert je devins très tendre ! 
Je pourrais dire sans façon : 
Vous n'

---

In [381]:
char, no_char = split_by_regex(fnames, REGICES["character"], orig_file=True)

with: 893
without: 190


In [387]:
# fnames_it = iter(char)
fnames_it = iter(no_char)

In [392]:
fname = next(fnames_it)
print_file(fname, whole_file=True)

count-2773495-Le_Minotaure.txt
------------------------------

Le Minotaure
 
de
 
Marcel Aymé
 
A Paris, de nos jours. 
 
Le décor représente une perspective de deux salons en enfilade, peints sur une toile de fond et meublés avec recherche. Complétant le décor de la toile, de vrais meubles prolongent l'un des salons sur la scène. Au premier plan, vers la droite, un tracteur rouge, l'avant tourné au public. Porte à droite, porte à gauche.
 
MOURLON, quarante ans, vêtu d'une combinaison de mécanicien est assis sur le siège du tracteur.
GERARD FORESTIER, quarante-cinq ans, vêtu avec une élégance sévère, se tient au milieu de la pièce et considère le tracteur en fermant à demi les yeux.
 
MOURLON
Je l'ai mis juste dans la diagonale, mais si vous voulez, je peux le redresser, c'est facile.
GERARD
Non, l'inclinaison est bonne. Ce que vous pourriez faire, c'est reculer un peu... Quoique... Attendez... Je me demande...
(Un silence.)
MOURLON
J'arrête le moteur ?
GERARD
Non... Une seconde, vou

---

In [310]:
caps_word, no_caps_word = split_by_regex(fnames, REGICES["caps_word"])

with: 1035
without: 48


In [311]:
# fnames_it = iter(caps_word)
fnames_it = iter(no_caps_word)

In [332]:
fname = next(fnames_it)
print_file(fname)

count-3109611-Chacun_a_sa_facon.txt
-----------------------------------




---

# TRIM ENTIRE FILE
# remove lines:
Traduction de

Collaborateur

# avertissement:

count-3141023-Ce_soir_on_improvise.txt

# list of chars without title: 

count-3013840-Menage_moderne.txt

count-2843262-La_grande_voix_de_la_presse.txt